In [ ]:
from methods.llm import get_vllm_endpoint_func, get_claude_response
POD_ID = "<INPUT>"
get_endpoint_response = get_vllm_endpoint_func("", POD_ID)

Could not load vllm class, check CUDA support and GPU RAM size
:: Sending warmup message to initialize the server ...
:: Server initialized successfully!


In [ ]:
# Test on meta_api fuctionals
from methods.meta_api import nodes_from_api
api_name = "langgraph"
doc_nodes, guide_nodes = await nodes_from_api(api_name, fast_response=get_endpoint_response, slow_response=get_claude_response, max_links=2) # Takes 2 mins to scrape langgraph

In [ ]:
# Try to get into this documentation instead for better scrawling
# https://python.langchain.com/v0.1/docs/use_cases/web_scraping/

In [7]:
%%timeit -n 3 -r 1
import http
import json
import urllib

from bs4 import BeautifulSoup
import httpx

def _search_google(query: str):
    """
    Use Serper API to search Google for information

    Args:
        query (str): The search query

    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": query})
    headers = {
        "X-API-KEY": os.environ["SERPER_API_KEY"],
        "Content-Type": "application/json",
    }

    try:
        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))
    except Exception as e:
        print(f"Error occurred during API request: {str(e)}")
        return {}
    finally:
        conn.close()


def get_content(link):
    res = httpx.get(link, follow_redirects=True)
    res.raise_for_status()
    url = str(res.url)

    return res.text, url

link = _search_google(f"{api_name} python docs")["organic"][0]["link"]
domain = urllib.parse.urlparse(link).netloc
try:
    content, url = get_content(link)
except httpx.HTTPStatusError as e:
    print(f"Error occurred during request: {str(e)}")

htmls = [content]
links = [url]
ptr = 0
print("Scraping links")
while ptr < len(links):
    soup = BeautifulSoup(htmls[ptr], "html.parser")
    for a in soup.find_all("a", href=True):
        href = a["href"]
        new_link = urllib.parse.urljoin(links[ptr], href.split("#")[0])
        parse = urllib.parse.urlparse(new_link)
        if (
            links[0] in new_link
            and new_link not in links
            and new_link + "/" not in links
            and parse.scheme in ["http", "https"]
        ):
            try:
                content, url = get_content(new_link)
                htmls.append(content)
                links.append(url)
            except httpx.HTTPStatusError as e:
                print(f"Error occurred during request: {str(e)}")
    ptr += 1

Scraping links
Scraping links
Scraping links
42 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


In [19]:
import http
import json
import urllib
import asyncio

from bs4 import BeautifulSoup
import httpx

limits = httpx.Limits(max_keepalive_connections=None, max_connections=None)
client = httpx.AsyncClient(timeout=None)

def _search_google(query: str):
    """
    Use Serper API to search Google for information

    Args:
        query (str): The search query

    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": query})
    headers = {
        "X-API-KEY": os.environ["SERPER_API_KEY"],
        "Content-Type": "application/json",
    }

    try:
        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))
    except Exception as e:
        print(f"Error occurred during API request: {str(e)}")
        return {}
    finally:
        conn.close()


async def aget_content(link, client):
    res = await client.get(link, follow_redirects=True)
    if res.status_code == 200:
        url = str(res.url)
        return res.text, url
    else:
        return "", ""

webpages = []
visit = set()

async def recursive_search(link, client, ref_url=None):
    content, url = await aget_content(link, client)
    if ref_url is None:
        ref_url = url
    if url and content:
        webpages.append((url, content))
        visit.add(url)
    soup = BeautifulSoup(content, "html.parser")
    new_links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        new_link = urllib.parse.urljoin(link, href.split("#")[0])
        parse = urllib.parse.urlparse(new_link)
        if (
            ref_url in new_link
            and new_link not in visit
            and new_link + "/" not in visit
            and parse.scheme in ["http", "https"]
        ):
            visit.add(new_link)
            new_links.append(new_link)
    await asyncio.gather(*[recursive_search(link, client, ref_url) for link in new_links])
    

link = _search_google(f"{api_name} python docs")["organic"][0]["link"]
domain = urllib.parse.urlparse(link).netloc

await recursive_search(link, client)

await client.aclose()

In [ ]:
# test case || evaluation || 
# - custom metric 
# - 